In [ ]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 880 kB 19.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=d32c7bee2caf09a277d52b7689bd520d401d7efc6f67e2d14758b6394f665950
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [1]:
import torch
import torch.nn as nn

import os

import torchtext.transforms as T
from torchtext.data.utils import get_tokenizer

from torchtext.vocab import vocab
from collections import OrderedDict

from torch.utils.data import Dataset, DataLoader

In [ ]:
emb = nn.Embedding(10, 5)

In [ ]:
a = torch.randint(0,10,(2,10),dtype=torch.int64)
emb(a)

tensor([[[ 1.2831,  0.3242,  0.9459, -1.7871,  0.2675],
         [-0.1369,  0.1705, -0.0410,  1.3350,  1.3271],
         [-1.0412,  1.2759, -0.1053, -0.0850, -1.1732],
         [-1.8657,  0.0713,  0.1171,  0.2328,  1.4423],
         [ 1.2831,  0.3242,  0.9459, -1.7871,  0.2675],
         [-0.1800, -0.8466, -0.1904, -1.0192,  2.6787],
         [-1.0412,  1.2759, -0.1053, -0.0850, -1.1732],
         [-1.0412,  1.2759, -0.1053, -0.0850, -1.1732],
         [-1.9008, -1.2194,  0.1945, -0.5162,  0.2388],
         [-1.9008, -1.2194,  0.1945, -0.5162,  0.2388]],

        [[-1.0412,  1.2759, -0.1053, -0.0850, -1.1732],
         [ 0.0444,  1.2466, -0.1850, -0.3290, -2.6041],
         [-0.1800, -0.8466, -0.1904, -1.0192,  2.6787],
         [-0.1369,  0.1705, -0.0410,  1.3350,  1.3271],
         [ 0.0444,  1.2466, -0.1850, -0.3290, -2.6041],
         [ 0.0444,  1.2466, -0.1850, -0.3290, -2.6041],
         [-1.0412,  1.2759, -0.1053, -0.0850, -1.1732],
         [-1.9721, -1.0152,  0.3521,  0.2212, 

In [ ]:
!wget https://www.statmt.org/europarl/v7/fr-en.tgz
!tar xfz *.tgz
!mkdir -p data/training
!mv *.en *.fr data/training

--2022-11-05 17:56:43--  https://www.statmt.org/europarl/v7/fr-en.tgz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 202718517 (193M) [application/x-gzip]
Saving to: ‘fr-en.tgz’

fr-en.tgz           100%[===================>] 193.33M  1.54MB/s    in 1m 49s  

2022-11-05 17:58:32 (1.77 MB/s) - ‘fr-en.tgz’ saved [202718517/202718517]



In [ ]:
# TODO: need to clean and speed this up, but for now it works fine
def get_phrases(path, tokenizer, size):
    phrases = []
    en_tokenize = get_tokenizer(tokenizer, language='en')
    fr_tokenize = get_tokenizer(tokenizer, language='fr')
    # get tokenized dataset
    with open(f'{path}/europarl-v7.fr-en.en') as en:
        with open(f'{path}/europarl-v7.fr-en.fr') as fr:
            k = 0
            for phrase in zip(en.readlines(), fr.readlines()):
                en_phrase = en_tokenize(phrase[0].strip())
                fr_phrase = fr_tokenize(phrase[1].strip())
                phrases.append({
                    'en': en_phrase,
                    'fr': fr_phrase,
                })
                k+=1
                if k >= size: break
    return phrases

def get_vocab(phrases, special_tokens):
    vocab_freq_en = {}
    vocab_freq_fr = {}

    for phrase in phrases:
        for word in phrase['en']:
            vocab_freq_en[word] = vocab_freq_en.get(word, 0) + 1

        for word in phrase['fr']:
            vocab_freq_fr[word] = vocab_freq_fr.get(word, 0) + 1

    vocab_en_ = sorted(vocab_freq_en.keys(), key=lambda x: vocab_freq_en[x], reverse=True)
    vocab_fr_ = sorted(vocab_freq_fr.keys(), key=lambda x: vocab_freq_fr[x], reverse=True)

    

    vocab_en = vocab_en_[:15_000] + special_tokens
    vocab_fr = vocab_fr_[:15_000] + special_tokens

    vocab_en = vocab(OrderedDict([(word, 1) for word in vocab_en]))
    vocab_en.set_default_index(-1)
    vocab_fr = vocab(OrderedDict([(word, 1) for word in vocab_fr]))
    vocab_fr.set_default_index(-1)

    return vocab_en, vocab_fr


def to_idx(phrases, vocab, special_tokens):
    max_len = 0
    idx_phrases = []
    vocab_en, vocab_fr = vocab
    for phrase in phrases:
        phrase['en'] = special_tokens[:1] + phrase['en'] + special_tokens[1:2]
        phrase['fr'] = special_tokens[:1] + phrase['fr'] + special_tokens[1:2]
        if max_len < len(phrase['en']):
            max_len = len(phrase['en'])
        if max_len < len(phrase['fr']):
            max_len = len(phrase['fr'])

    for phrase in phrases:
        en_phrase = phrase['en']
        fr_phrase = phrase['fr']

        en_phrase_idx = T.VocabTransform(vocab_en)(en_phrase)
        fr_phrase_idx = T.VocabTransform(vocab_fr)(fr_phrase)

        idx_phrases.append({
            'en': en_phrase_idx,
            'fr': fr_phrase_idx,
        })
    return idx_phrases, max_len

class WMT2014(Dataset):
    def __init__(self, path, special_tokens, show_idx=True):
        self.path = path
        self.show_idx = show_idx
        self.special_tokens = special_tokens
        self.phrases = get_phrases(self.path, 'moses', 100_000)
        self.vocab = get_vocab(self.phrases, self.special_tokens)
        self.idx_phrases, self.max_len = to_idx(self.phrases, self.vocab, self.special_tokens)

    def __getitem__(self, idx) :
        if self.show_idx:
            padded_src = torch.zeros(self.max_len, dtype=torch.int32)
            padded_tgt = torch.zeros(self.max_len, dtype=torch.int32)
            src = torch.tensor(self.idx_phrases[idx]['en'])
            tgt = torch.tensor(self.idx_phrases[idx]['fr'])

            padded_src[:src.shape[0]] = src
            padded_tgt[:tgt.shape[0]] = tgt

            return padded_src, padded_tgt
        else:
            src = self.phrases[idx]['en']
            tgt = self.phrases[idx]['fr']
            return src, tgt
        

        return src, tgt

    def __len__(self):
        return len(self.phrases)

In [ ]:
ds = WMT2014('data/training', ['<SOS>', '<EOS>', '<UNK>'])

In [ ]:
dl = DataLoader(ds, batch_size=128, shuffle=True)

In [242]:
# TODO: add masking
class MultiHeadAttention(nn.Module):
    def __init__(self, seq_len, d_model, num_heads, mask=False):
        super(MultiHeadAttention, self).__init__()

        self.d_model = d_model
        self.seq_len = seq_len
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_o = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def reshape(self, x):
        # [bs, seq_len, num_heads, d_k]
        out = x.reshape(*x.shape[:2], self.num_heads, self.d_k)
        # [bs, num_heads, seq_len, d_k]
        return out.permute(0, 2, 1, 3)

    def attention(self, q, k, v):
        scale = 1 / self.d_k**0.5
        out = q @ k.transpose(-2, -1) * scale
        out = self.softmax(out) @ v
        return out

    def forward(self, q, k, v):
        # [bs, num_heads, seq_len, d_k]
        q = self.reshape(self.w_q(q))
        k = self.reshape(self.w_k(k))
        v = self.reshape(self.w_v(v))

        # [bs, seq_len, num_heads, d_k]
        attn = self.attention(q, v, k).permute(0, 2, 1, 3)
        # [bs, seq_len, d_model]
        attn = attn.reshape(-1, self.seq_len, self.d_model)

        out = self.w_o(attn)

        return out

class PositionalEncoding(nn.Module):
    def __init__(self, seq_len, emb_dim):
        super(PositionalEncoding, self).__init__()

        self.pe = torch.zeros(seq_len, emb_dim)
        pos = torch.arange(seq_len).unsqueeze(1)
        i = torch.arange(0, emb_dim, 2)
        div = 1 / torch.pow(10000, i / emb_dim)
        self.pe[:, 0::2] = torch.sin(pos*div)
        self.pe[:, 1::2] = torch.cos(pos*div)

        # give it batch dim
        self.pe = self.pe.unsqueeze(0)

    def forward(self, emb):
        return emb + self.pe

class EncoderBlock(nn.Module):
    def __init__(self, seq_len, d_model, num_heads):
        super(EncoderBlock, self).__init__()

        norm_shape = (seq_len, d_model)

        self.attn = MultiHeadAttention(seq_len, d_model, num_heads)
        self.ln_1 = nn.LayerNorm(norm_shape)

        self.ff = nn.Sequential(
            nn.Linear(d_model, 2048),
            nn.ReLU(),
            nn.Linear(2048, d_model),
        )

        self.ln_2 = nn.LayerNorm(norm_shape)
    
    def forward(self, src):
        attn = self.attn(src, src, src)
        out_1 = self.ln_1(src + attn)

        ff = self.ff(out_1)
        out_2 = self.ln_2(out_1 + ff)

        return out_2

class DecoderBlock(nn.Module):
    def __init__(self, seq_len, d_model, num_heads):
        super(DecoderBlock, self).__init__()
        
        norm_shape = (seq_len, d_model)

        self.attn_1 = MultiHeadAttention(seq_len, d_model, num_heads)
        self.ln_1 = nn.LayerNorm(norm_shape)

        self.attn_2 = MultiHeadAttention(seq_len, d_model, num_heads)
        self.ln_2 = nn.LayerNorm(norm_shape)

        self.ff = nn.Sequential(
            nn.Linear(d_model, 2048),
            nn.ReLU(),
            nn.Linear(2048, d_model),
        )
        self.ln_3 = nn.LayerNorm(norm_shape)

    # TODO: find better solutions for decoder inputs
    def forward(self, args):
        enc = args[0]
        tgt = args[1]

        attn_1 = self.attn_1(tgt, tgt, tgt)
        out_1 = self.ln_1(enc + attn_1)

        attn_2 = self.attn_2(enc, enc, out_1)
        out_2 = self.ln_2(out_1 + attn_2)

        ff = self.ff(out_2)
        out_3 = self.ln_3(out_2 + ff)

        return enc, out_3

class Transformer(nn.Module):
    def __init__(self, seq_len, vocab_len, d_model, num_blocks, num_heads):
        super(Transformer, self).__init__()

        self.inp_emb = nn.Embedding(vocab_len, d_model)
        self.out_emb = nn.Embedding(vocab_len, d_model)
        self.pos_enc = PositionalEncoding(seq_len, d_model)

        self.enc = nn.Sequential(
            *[EncoderBlock(seq_len, d_model, num_heads) for _ in range(num_blocks)]
        )

        self.dec = nn.Sequential(
            *[DecoderBlock(seq_len, d_model, num_heads) for _ in range(num_blocks)]
        )

        self.out = nn.Sequential(
            nn.Linear(d_model, vocab_len),
            nn.Softmax(dim=-1)
        )

    def forward(self, src, tgt):
        inp_emb = self.pos_enc(self.inp_emb(src))
        enc = self.enc(inp_emb)

        out_emb = self.out_emb(tgt)
        _, dec = self.dec((enc, out_emb))

        out = self.out(dec)

        return out

In [249]:
s = 10
v = 128
bs = 16
nh = 8
dm = 512
nb = 6

t = Transformer(s, v, dm, nb, nh)

src = torch.randint(v, (bs, s))
tgt = torch.randint(v, (bs, s))

# inp_emb = nn.Embedding(v, dm)(src)
# pe = PositionalEncoding(s, dm)(inp_emb)
# enc = EncoderBlock(s, dm, nh)
# dec = DecoderBlock(s, dm, nh)
# dec((enc(pe), pe)).shape

out = t(src, tgt)
print(out.shape)
out.argmax(dim=-1)

torch.Size([16, 10, 128])


tensor([[122,   7, 101,  62,  50, 103,  91, 103,   8, 122],
        [124,  97, 103,  81,  41, 103,  81, 122,  91, 122],
        [ 26, 103,  11, 122,  86,  44,  59,   1,  53, 122],
        [ 23,  50,  25,  91, 102, 103, 122,  26,  46, 103],
        [ 41,  26,  26,  26,   1,  41,  85,  64, 111,  41],
        [103,  54,  26,  86,   1,  50,  68,  23, 103,  26],
        [ 50,  31,  50, 122, 105,  52,  50,  50,   5,  50],
        [ 50, 103,  26, 122,  11, 118, 101, 122, 122, 122],
        [103,   1,  97, 124,  50,  50, 100,  26, 103,  26],
        [103,  54,  52,  50, 122,  50,  54,  50, 122,  54],
        [ 26,  26,  26,  26,  74, 111, 103,  50, 111, 122],
        [ 31,  52,  97,  31, 103,  91,  54,  52,   5,  91],
        [ 50,  46, 100, 100,   4,  12,  26, 103, 107,  26],
        [122, 122, 122, 114, 122,  17,  59,  50,  54,   7],
        [103,  58, 103, 103,  62,  62, 100, 101,  58, 122],
        [ 85,  50, 122,  11,   1,  72, 122,  41, 122, 112]])